В этом домашнем задании будем оценивать склонность клиента банка откликнуться или не откликнуться на предложение.

Существует очень простой метод машинного обучения - [метод ближайших соседей, KNN](https://ru.wikipedia.org/wiki/Метод_k-ближайших_соседей) (для понимания метода понадобится прочесть первые два абзаца по ссылке). У него нет весов, поэтому он как правило плохо работает, но для сравнения с другими методами нам вполне подойдет.

В этом домашнем задании вам предлагается решить задачу с помощью KNN и логистической регрессии и сравнить результаты.

In [ ]:
# подключим библиотеки
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# считаем данные
data = pd.read_csv('ClientsData.csv')

In [ ]:
data.head()

In [ ]:
data.shape

В этом ноутбуке нам придется подбирать гиперпараметры модели, а ещё и порог, поэтому чтобы не переобучиться, разобъем данные на трейн, валидацию и тест.

*  обучать модели будем на тренировочных данных
*  подбирать необходимые величины - по валидации
*  оценивать качество - на тесте

In [ ]:
# разделим данные на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split

X = data.drop('TARGET', axis=1)
y = data['TARGET']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=123)
Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, train_size=0.7, random_state=123)

### KNN

Обучите на тренировочных данных KNN с параметрами по умолчанию и выведите матрицу ошибок, а также значение метрик precision и recall на тестовых данных.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

In [ ]:
# ваш код здесь

Теперь с помощью GridSearchCV подберите оптимальное число соседей k в методе (в sklearn параметр называется n_neighbors). Ищите значения в диапазоне от 1 до 15.

Для наилучшего k обучите модель на тренировочных данных и выведите матрицу ошибок, а также значение метрик precision и recall на тестовых данных.

In [ ]:
# ваш код здесь

Какой вывод можно сделать?

Напишите ваш вывод о качестве работы модели текстом.

### LogReg

Обучите логистическую регрессию с параметрами по умолчанию и посмотрите на метрики.

Везде дальше при оценке метрик надо выводить confusion_matrix, precision и recall.

In [ ]:
from sklearn.linear_model import LogisticRegression

# ваш код здесь

Наша цель - найти как можно больше клиентов, кто откликнется на предложение.

Предскажите вероятности классов с помощью обученной логистической регрессии на тестовых данных, и выведите вероятности положительного класса для первых 10 объектов.

Глядя на полученные вероятности, попробуйте объяснить, почему вы получили именно такую матрицу ошибок и такие значения точности с полноты.

In [1]:
# ваш код здесь

In [ ]:
# ваше объяснение здесь

Давайте уточним нашу цель. Пусть нам нужно найти как можно больше клиентов, кто откликнется на предложение, то есть максимизировать полноту (recall). 

При этом нам хочется, чтобы точность модели (precision) не была очень низкой. Заказчик посмотрел на результаты работы KNN и потребовал, чтобы точность была не ниже, чем у KNN: $precision \geq 0.13$.

Давайте будем изменять порог для перевода вероятности в классы так, чтобы:
*   максимизировать значение recall
*   при условии, что $precision \geq 0.13$.

Если мы будем подбирать порог по тестовой выборке, то мы по сути обучимся на ней и, значит, переобучимся. Это плохо. 

Поэтому прескажите вероятности на валидационной выборке и подберите порог по валидационной выборке (Xval, yval), а затем посмотрите, какое качество для найденного порога вы получите на тестовых данных.

In [ ]:
probs_val = lr.predict_proba(Xval)[:,1]

max_recall = -1
thr = -1
prec = -1

for threshold in np.arange(0.05, 0.25, 0.001):
    # для каждого значения порога переведите вероятности в классы
    # посчитайте метрики

# напечатайте порог, для которого получается максимальная полнота, при precision >= 0.13

In [ ]:
# по найденному порогу переведите вероятности в классы на тесте и напечатайте метрики

Сделайте вывод. Смогли ли мы с помощью подбора порога добиться бОльшего значения recall, чем у KNN? 

In [ ]:
# ваш вывод здесь

А ещё можно подбирать значение гиперпараметра C у логистической регрессии для улучшения качества предсказания. 

1) Обучите для значений C из диапазона [0.05, 0.15, 0.25, ...., 10.05] логистическую регрессию (на тренировочных данных).

2) Для каждой из обученных моделей во внутреннем цикле подберите оптимальный порог (как в предыдущем задании) - на валидационных данных.



В качестве результата выведите значение C и порога для модели, которая дает наилучшие значения метрик (наибольший recall при ограничении на $precision \geq 0.13$).

Также напечатайте полученные метрики (матрицу ошибок, точность и полноту) для лучшей модели - на тестовых данных.

In [ ]:
regs = []
recalls = []
thresholds = []
precisions = []

for reg in np.arange(0.001, 0.1, 0.01):

    # обучите логистическую регрессию с C=reg

    max_recall = -1
    thr = -1
    prec = -1

    for threshold in np.arange(0.05, 0.25, 0.001):
        # подберите оптимальный порог как в задании выше

    recalls.append(max_recall)
    thresholds.append(thr)
    precisions.append(prec)
    regs.append(C)

In [ ]:
# выведите значения C, precision, recall, threshold для наилучшей по заданным условиям модели

In [ ]:
# с помощью найденных C и threshold обучите модель на тренировочных данных, сделайте предсказание на тесте и по найденному порогу получите классы.
# напечатайте метрики

Влияет ли изменение гиперпараметра C на качество модели (и, соответственно, метрики) в этой задаче?

In [ ]:
# ваш вывод здесь

Удалось ли при помощи логистической регрессии и подбора порога превзойти качество метода ближайших соседей в этой задаче? 

Смогли ли мы при помощи этой модели получить высокий recall при ограничениях, поставленных заказчиком?

In [ ]:
# ваш вывод здесь